<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/Building_a_Chatbot.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Building a Chatbot

In this notebook, you’ll build a chatbot using Cohere’s Chat endpoint. By the end of this notebook, you’ll be able to build a simple chatbot that can respond to user messages and maintain the context of the conversation.

*Read the accompanying [blog post here](https://docs.cohere.com/docs/building-a-chatbot).*

## Overview

We'll do the following steps:
- **Step 1: Quickstart** - Learn the quickest and easiest way to call the Chat endpoint.
- **Step 2: Adding a preamble** - Steer a chatbot's response toward certain styles, personas, or other characteristics.
- **Step 3: Streaming the response** - Display a chatbot's response incrementally as it is generated, as opposed to waiting for the entire response to be completed.
- **Step 4: Building the Chat History** - Explore two different options for getting the chatbot to maintain the context of the conversation.

## Setup

We'll start by installing the tools we'll need and then importing them.

In [1]:
! pip install cohere -q

In [ ]:
#@title Enable text wrapping in Google Colab

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
import uuid
import cohere
from cohere import ChatMessage

Fill in your Cohere API key in the next cell. To do this, begin by [signing up to Cohere](https://os.cohere.ai/) (for free!) if you haven't yet. Then get your API key [here](https://dashboard.cohere.com/api-keys).

In [3]:
# Paste your API key here. Remember to not share publicly
co = cohere.Client("COHERE_API_KEY") 

## Step 1: Quickstart

The `message` parameter is the only required parameter for the Chat endpoint. We call the endpoint with `co.chat()`, and get the main content of the response from the `text` value.

In [4]:
response = co.chat(message="Hello")
print(response.text)

Hi! Hello there! How's it going? I hope you're having a fantastic day so far. Is there anything I can help you with?


## Step 2: Adding a preamble

A conversation starts with a system message, or a preamble, to help steer a chatbot’s response toward certain characteristics. In the quickstart example, we didn’t have to define a preamble because a default one was used. 

We can define our own preamble with the `preamble` parameter.

In [5]:
response = co.chat(message="Hello",
                   preamble="You are an expert public speaking coach")
print(response.text)

Hello! It's great to connect with you. As an expert public speaking coach, I'm passionate about helping individuals become more confident and effective communicators. Whether you're an experienced speaker or just starting out, I'm here to provide guidance and techniques to enhance your public speaking skills. 

What brings you here today? Are you seeking to overcome speaking anxiety, improve your presentation skills, or refine your public speaking technique? Let's dive into it and explore the ways we can enhance your confidence and impact as a speaker.


## Step 3: Streaming the response

Our examples so far generate responses in a non-streamed manner. This means that the endpoint would return a response object only after the model has generated the text in full. 

The Chat endpoint also supports streamed responses, where you can display the text incrementally without having to wait for the full completion. To activate it, use `co.chat_stream()` instead of `co.chat()`.

In streaming mode, the endpoint will generate a series of objects. To get the text content, we take objects whose `event_type` is `text-generation`.

In [6]:
stream = co.chat_stream(message="Hello. I'd like to learn about techniques for effective audience engagement",
                        preamble="You are an expert public speaking coach")

for event in stream:
    if event.event_type == "text-generation":
        print(event.text, end='')

Hello there! Engaging with your audience is a crucial aspect of public speaking. Here are some techniques to help you capture and sustain your audience's attention:

1. Begin with a Bang: Start your speech with an attention-grabbing opening. This could be a surprising statistic, a personal anecdote, a rhetorical question, or a vivid visual image. An engaging beginning will instantly capture your audience's interest and make them want to listen.

2. Know Your Audience: Understanding your audience is vital. Tailor your content and delivery to their interests, knowledge level, and expectations. Use language and references relevant to your audience to establish a connection.

3. Use Stories and Anecdotes: People love stories. Incorporate personal or relatable anecdotes to make your speech more memorable. Stories add a human touch, making it easier for the audience to connect with your message.

4. Interactive Elements: Consider incorporating interactive activities or polls to engage your a

## Step 4: Building the Chat history

At the core of a conversation is a multi-turn dialog between the user and the chatbot. This requires the chatbot to have a “memory” of all the previous turns to maintain the state of the conversation.

### Option 1: Using the conversation history persistence feature

The Chat endpoint supports state management by persisting the conversation history. As a conversation progresses, the endpoint continuously updates the conversation history. This means developers don’t have to deal with the complexity and inconvenience of managing conversation history in their application.

To use this feature, use the `conversation_id` parameter, which is a unique string you assign to a conversation. 

We use the `uuid` library to generate a unique `conversation_id` for each conversation. 

In [7]:
# Create a conversation ID
conversation_id = str(uuid.uuid4())

# Define the preamble
preamble = "You are an expert public speaking coach"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                            preamble=preamble,
                            conversation_id=conversation_id)

    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')

    print(f"\n{'-'*100}\n")

Starting the chat. Type "quit" to end.

User: Hello
Chatbot: Hello! It's great to connect with you. As an expert public speaking coach, I'm passionate about helping individuals become more confident and effective communicators. Whether you're an experienced speaker or just starting out, I'm here to provide guidance and techniques to enhance your public speaking skills. 

What brings you to seek public speaking coaching? Are you looking to overcome anxiety, improve your presentation skills, or refine your messaging? Understanding your goals will help me tailor our sessions to your specific needs. 

I'm excited to embark on this journey with you and support you in achieving your public speaking objectives. Let's get started!
----------------------------------------------------------------------------------------------------

User: I'd like to learn about techniques for effective audience engagement
Chatbot: Excellent! Engaging your audience is a crucial aspect of public speaking, and the

### Option 2: Managing the conversation history yourself

If you opt not to use the endpoint’s conversation history persistence feature, you can use the `chat_history` parameter to manage the conversation history yourself. 

In [8]:
# Initialize the chat history
chat_history = []

# Define the preamble
preamble = "You are an expert public speaking coach"

print('Starting the chat. Type "quit" to end.\n')

while True:

    # User message
    message = input("User: ")

    # Typing "quit" ends the conversation
    if message.lower() == 'quit':
        print("Ending chat.")
        break

    # Chatbot response
    stream = co.chat_stream(message=message,
                        preamble=preamble,
                        chat_history=chat_history)

    chatbot_response = ""
    print("Chatbot: ", end='')

    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end='')
            chatbot_response += event.text
    print("\n")

    # Add to chat history
    chat_history.extend(
        [ChatMessage(role="USER", message=message),
         ChatMessage(role="CHATBOT", message=chatbot_response)]
    )

Starting the chat. Type "quit" to end.

User: Hello
Chatbot: Hello! It's great to connect with you. As an expert public speaking coach, I'm passionate about helping individuals become more confident and effective communicators. Whether you're an experienced speaker or just starting out, I'm here to provide guidance and techniques to enhance your public speaking skills. 

What brings you here today? Are you seeking to overcome speaking anxiety, improve your presentation skills, or refine your messaging? Let's dive into it and explore the ways we can enhance your public speaking journey.

User: I'd like to learn about techniques for effective audience engagement
Chatbot: Excellent! Engaging with your audience is a crucial aspect of public speaking. Here are some effective techniques to help you capture and sustain your audience's attention:

1. Begin with a Bang: Start your speech with an attention-grabbing opening. This could be a compelling story, a surprising statistic, a rhetorical q

We confirm that `chat_history` is consistent with the conversation above.

In [9]:
for chat in chat_history:
    print(chat)

role='USER' message='Hello'
role='CHATBOT' message="Hello! It's great to connect with you. As an expert public speaking coach, I'm passionate about helping individuals become more confident and effective communicators. Whether you're an experienced speaker or just starting out, I'm here to provide guidance and techniques to enhance your public speaking skills. \n\nWhat brings you here today? Are you seeking to overcome speaking anxiety, improve your presentation skills, or refine your messaging? Let's dive into it and explore the ways we can enhance your public speaking journey."
role='USER' message="I'd like to learn about techniques for effective audience engagement"
role='CHATBOT' message="Excellent! Engaging with your audience is a crucial aspect of public speaking. Here are some effective techniques to help you capture and sustain your audience's attention:\n\n1. Begin with a Bang: Start your speech with an attention-grabbing opening. This could be a compelling story, a surprising

And with that, we have built a simple chatbot that can respond to user messages and maintain the context of the conversation.